# Post-Hoc Analysis: Hybrid CNN + Attention for EEG Decoding

Run this after `scripts/run_all_ablations.sh` completes.
Generates publication-ready figures and statistical tests.

In [ ]:
import sys
sys.path.insert(0, '..')

import json
import numpy as np
from pathlib import Path

from utils.statistics import aggregate_results, compare_models
from visualizations.summary_plots import (
    plot_ablation_comparison,
    plot_training_curves,
    plot_confusion_matrices,
    generate_results_table,
)

In [ ]:
LOG_DIR = '../results/logs'
FIG_DIR = '../results/figures'

# Load all results
models = {
    'EEGNet': aggregate_results(LOG_DIR, 'eegnet_none'),
    'DeepConvNet': aggregate_results(LOG_DIR, 'deep_convnet_none'),
    'Hybrid (no attn)': aggregate_results(LOG_DIR, 'hybrid_cnn_none'),
    'Hybrid + SE': aggregate_results(LOG_DIR, 'hybrid_cnn_se'),
    'Hybrid + CBAM': aggregate_results(LOG_DIR, 'hybrid_cnn_cbam'),
    'Hybrid + MHA': aggregate_results(LOG_DIR, 'hybrid_cnn_mha'),
}

print('Loaded results:')
for name, res in models.items():
    if 'accuracy' in res:
        acc = res['accuracy']
        print(f'  {name:25s}: {np.mean(acc):.4f} ± {np.std(acc):.4f} (n={len(acc)})')

In [ ]:
# Box plot comparison
plot_ablation_comparison(
    models,
    metric='accuracy',
    title='Ablation Study — Test Accuracy (BCI-IV-2a)',
    save_path=f'{FIG_DIR}/ablation_accuracy.png',
)

In [ ]:
# Statistical tests
comparisons = [
    ('hybrid_cnn_se', 'eegnet_none'),
    ('hybrid_cnn_cbam', 'eegnet_none'),
    ('hybrid_cnn_mha', 'eegnet_none'),
    ('hybrid_cnn_se', 'hybrid_cnn_none'),
]

for a, b in comparisons:
    result = compare_models(LOG_DIR, a, b)
    print(f'{a} vs {b}:')
    for k, v in result.items():
        print(f'  {k}: {v}')
    print()

In [ ]:
# Results table
generate_results_table(LOG_DIR, save_path=f'{FIG_DIR}/../results_table.txt')